In [1]:
import json
from pandas.io.json import json_normalize
from pprint import pprint as pprint
import requests
import sys
import urllib
import pandas as pd
import numpy as np
import folium
import os

from folium.plugins import HeatMap
from urllib.error import HTTPError
from urllib.parse import quote
from urllib.parse import urlencode

API_KEY= "pLqmLsYj1SsyQqgqFZ2Inn3AhOxuTOKVbwOu043-4O5snad4ido3E1XWuG5g_iVsZ6LvtqPusPoV_pBPJ6nILp5vpNH_RDSNVfyHJ3T693QsJNDqCt5oGqT_Jy6kWnYx"

# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'


DEFAULT_TERM = 'Restaurants'
DEFAULT_LOCATION = 'Los Angeles, CA'
SEARCH_LIMIT = 50




In [2]:
def request(host, path, api_key, url_params=None):

    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


In [3]:

def search_loc(api_key, term, location, offset):

    url_params = {
        'offset': offset,
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)

In [4]:

def search_coords(api_key, term, latitude, longitude, radius, offset):

    url_params = {
        'offset': offset,
        'term': term.replace(' ', '+'),
        'latitude': latitude,
        'longitude': longitude,
        'radius': radius,
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)

In [5]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[str(name[:-1])] = str(x)

    flatten(y)
    return out

In [6]:

def query_api_loc(term, location, offset):
    """ term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search_loc(API_KEY, term, location, offset)
    businesses = response.get('businesses')
    out = pd.DataFrame(businesses)
    return out


In [7]:
def query_api_coords(term, lat, long, radius, offset):
    response = search_coords(API_KEY, term, lat,long,radius, offset)
    #pprint(response)
    businesses = response.get('businesses')
    out = pd.DataFrame(businesses)
    return out

In [8]:
df= query_api_loc("bars", 'placentia', 0)
df2 = query_api_coords("bars", 33.8711, -117.8628, 8000, 0)
#df=df.drop_duplicates(subset = ['id','longitude','latitude'])
#df =df[df.is_closed == False]

Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...


In [9]:
df.head()


,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...","{'latitude': 33.889985, 'longitude': -117.84723}",(714) 961-9396,1045.223407,kellys-korner-tavern-placentia,https://s3-media4.fl.yelpcdn.com/bphoto/OE5EG1...,False,"{'address1': '907 E Yorba Linda Blvd', 'addres...",Kelly's Korner Tavern,+17149619396,$$,4.5,230,"[delivery, pickup]",https://www.yelp.com/biz/kellys-korner-tavern-...
1,"[{'alias': 'sportsbars', 'title': 'Sports Bars...","{'latitude': 33.88991, 'longitude': -117.84746}",(714) 646-9569,1018.122000,rems-lounge-placentia-3,https://s3-media3.fl.yelpcdn.com/bphoto/I7bJth...,False,"{'address1': '909 E Yorba Linda Blvd', 'addres...",Rem's Lounge,+17146469569,$$,4.0,86,[],https://www.yelp.com/biz/rems-lounge-placentia...
2,"[{'alias': 'breweries', 'title': 'Breweries'}]","{'latitude': 33.8624287319832, 'longitude': -1...",(714) 996-6258,3318.147235,the-bruery-placentia-2,https://s3-media4.fl.yelpcdn.com/bphoto/Peenff...,False,"{'address1': '717 Dunn Way', 'address2': '', '...",The Bruery,+17149966258,$$,4.5,872,[],https://www.yelp.com/biz/the-bruery-placentia-...
3,"[{'alias': 'sportsbars', 'title': 'Sports Bars...","{'latitude': 33.8670486, 'longitude': -117.819...",(714) 528-7000,3261.568923,clubhouse-grille-and-sports-bar-placentia-2,https://s3-media2.fl.yelpcdn.com/bphoto/AhKknE...,False,"{'address1': '2053 E Orangethorpe Ave', 'addre...",Clubhouse Grille & Sports Bar,+17145287000,$,4.0,65,[],https://www.yelp.com/biz/clubhouse-grille-and-...
4,"[{'alias': 'sportsbars', 'title': 'Sports Bars'}]","{'latitude': 33.9041529687571, 'longitude': -1...",(714) 528-9888,2704.502411,group-therapy-pub-placentia,https://s3-media2.fl.yelpcdn.com/bphoto/SxfkqM...,False,"{'address1': '1265 E Imperial Hwy', 'address2'...",Group Therapy Pub,+17145289888,$,3.5,36,[],https://www.yelp.com/biz/group-therapy-pub-pla...


In [10]:
df.columns

Index(['categories', 'coordinates', 'display_phone', 'distance', 'id',
       'image_url', 'is_closed', 'location', 'name', 'phone', 'price',
       'rating', 'review_count', 'transactions', 'url'],
      dtype='object')

In [11]:
df['Latitude']=df['coordinates'].apply(lambda x: x['longitude'])
df['Longitude']=df['coordinates'].apply(lambda x: x['latitude'])

In [12]:
df.head()

,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url,Latitude,Longitude
0,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...","{'latitude': 33.889985, 'longitude': -117.84723}",(714) 961-9396,1045.223407,kellys-korner-tavern-placentia,https://s3-media4.fl.yelpcdn.com/bphoto/OE5EG1...,False,"{'address1': '907 E Yorba Linda Blvd', 'addres...",Kelly's Korner Tavern,+17149619396,$$,4.5,230,"[delivery, pickup]",https://www.yelp.com/biz/kellys-korner-tavern-...,-117.847230,33.889985
1,"[{'alias': 'sportsbars', 'title': 'Sports Bars...","{'latitude': 33.88991, 'longitude': -117.84746}",(714) 646-9569,1018.122000,rems-lounge-placentia-3,https://s3-media3.fl.yelpcdn.com/bphoto/I7bJth...,False,"{'address1': '909 E Yorba Linda Blvd', 'addres...",Rem's Lounge,+17146469569,$$,4.0,86,[],https://www.yelp.com/biz/rems-lounge-placentia...,-117.847460,33.889910
2,"[{'alias': 'breweries', 'title': 'Breweries'}]","{'latitude': 33.8624287319832, 'longitude': -1...",(714) 996-6258,3318.147235,the-bruery-placentia-2,https://s3-media4.fl.yelpcdn.com/bphoto/Peenff...,False,"{'address1': '717 Dunn Way', 'address2': '', '...",The Bruery,+17149966258,$$,4.5,872,[],https://www.yelp.com/biz/the-bruery-placentia-...,-117.879275,33.862429
3,"[{'alias': 'sportsbars', 'title': 'Sports Bars...","{'latitude': 33.8670486, 'longitude': -117.819...",(714) 528-7000,3261.568923,clubhouse-grille-and-sports-bar-placentia-2,https://s3-media2.fl.yelpcdn.com/bphoto/AhKknE...,False,"{'address1': '2053 E Orangethorpe Ave', 'addre...",Clubhouse Grille & Sports Bar,+17145287000,$,4.0,65,[],https://www.yelp.com/biz/clubhouse-grille-and-...,-117.819919,33.867049
4,"[{'alias': 'sportsbars', 'title': 'Sports Bars'}]","{'latitude': 33.9041529687571, 'longitude': -1...",(714) 528-9888,2704.502411,group-therapy-pub-placentia,https://s3-media2.fl.yelpcdn.com/bphoto/SxfkqM...,False,"{'address1': '1265 E Imperial Hwy', 'address2'...",Group Therapy Pub,+17145289888,$,3.5,36,[],https://www.yelp.com/biz/group-therapy-pub-pla...,-117.841555,33.904153


In [15]:
#LA SAMPLE
LA_sample = query_api_coords("food", 33.8711, -117.8628, 40000, 0)
offset = 50
count = 1000

while offset <= count:
    df = query_api_coords("bars", 33.8711, -117.8628, 40000, offset)
    LA_sample = LA_sample.append(df, ignore_index=True)
    LA_sample = pd.concat([LA_sample, df], ignore_index=True)
    offset += 50

LA_sample.to_csv('LA_SAMPLE.csv', sep='\t', encoding='utf-8')


Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying h

In [21]:
LA_sample.head()
LA_sample.to_csv('LA_SAMPLE.csv', sep='\t', encoding='utf-8')

In [14]:
LA_sample['Latitude']=df['coordinates'].apply(lambda x: x['latitude'])
LA_sample['Longitude']=df['coordinates'].apply(lambda x: x['longitude'])

KeyError: 'coordinates'

In [18]:
data = (np.random.normal(size=(100, 2)) *
        np.array([[.2, .2]]) +
        np.array([[34, -118]])).tolist()


#LA county bound box
#$$dW1184005$$eW1180919$$fN0342014$$gN0334213
#"Los Angeles","Los Angeles County","POLYGON((-118.951721 32.75004,-118.951721 34.823302,-117.646374 34.823302,-117.646374 32.75004,-118.951721 32.75004))","32.75004,-118.951721","34.823302,-118.951721","34.823302,-117.646374","32.75004,-117.646374","32.75004,-118.951721","34.823302","32.7500","-117.646374","-118.951721" "06","039","00277284","06039"
data = []
for index, row in LA_sample.iterrows():
    try:
        data.append([LA_sample.iloc[index]['coordinates']['latitude'],LA_sample.iloc[index]['coordinates']['longitude']])
    except(KeyError):
        print("fuck")
data

[[33.889376956228, -117.870076149702],
 [33.873722, -117.8897449],
 [33.8771903, -117.8431781],
 [33.87821595389, -117.878588140011],
 [33.8742549, -117.9034041],
 [33.8732328828398, -117.879818014227],
 [33.8424195259131, -117.851911864822],
 [33.869873, -117.924963],
 [33.87257, -117.89025],
 [33.8743449, -117.890335],
 [33.8747525126103, -117.889207324384],
 [33.85715, -117.86749],
 [33.8732049, -117.8763828],
 [33.9092033822344, -117.849996131924],
 [33.8726450658277, -117.89023462683],
 [33.8704690233034, -117.871121648946],
 [33.8263001, -117.8359851],
 [33.8911057, -117.8734613],
 [33.8313913199616, -117.912403905831],
 [33.8763748981511, -117.879422307014],
 [33.8874697744823, -117.862867456132],
 [33.91195, -117.89024],
 [33.8705049, -117.9227446],
 [33.8357217, -117.9162517],
 [33.9112149, -117.8674685],
 [33.8709036, -117.9205492],
 [33.8744772762321, -117.885255142853],
 [33.8542053448348, -117.838509137897],
 [33.9112149, -117.8674685],
 [33.8269853, -117.9762487],
 [33.86

In [19]:
m = folium.Map([34, -118], tiles='stamentoner', zoom_start=9)

HeatMap(data).add_to(m)

m.save(os.path.join('results', 'Heatmap.html'))

m


In [23]:
LA = folium.Map(location=[34.0522, -118.2437],
                    zoom_start = 10) 
LA


In [24]:
la_county_edge = os.path.join('data', 'la_county_edge.json')

m = folium.Map(
    location=[33.5700, -118.2437],
    tiles='Stamen Toner',
    zoom_start=8
)


folium.GeoJson(
    la_county_edge,
    name='geojson'
).add_to(m)
#def search_grid(radius, lat, long)
#convert from R

#def search_box(lat1,long1, lat2,lat2) 
#lat1,long1 of sw corner
#lat2,long2 of ne corner


#formula for lat and long degrees to meters?

rad = 12750

lat_a=34.8300
long_a = -118.1275
lat_b = 34.7300
long_b = -117.9300


for x in range(0,6):
    folium.features.Circle(radius=rad,location=[lat_a-(.2*x), long_a], color ='gold',fill=False).add_to(m)
for x in range(0,5):
    folium.features.Circle(radius=rad,location=[lat_a-(.2*x), long_a+.4], color ='purple',fill=False).add_to(m)
for x in range(0,5):
    folium.features.Circle(radius=rad,location=[lat_a-(.2*x), long_a-.4], color ='blue',fill=False).add_to(m)
for x in range(0,5):
    if(x==4):
        folium.features.Circle(radius=rad,location=[lat_a-(.2*x), long_a-.8], color ='blue',fill=False).add_to(m)
    




for x in range(0,5):
    folium.features.Circle(radius=rad,location=[lat_b-(.2*x), long_b], color ='black',fill=False).add_to(m)
for x in range(0,6):
    folium.features.Circle(radius=rad,location=[lat_b-(.2*x), long_b-.4], color ='crimson',fill=False).add_to(m)
for x in range(0,4):
    if(x!=2):
        folium.features.Circle(radius=rad,location=[lat_b-(.2*x), long_b-.8], color ='green',fill=False).add_to(m)    
m.save('LA county yelp search hex-circle grid.html') 
m

In [25]:
rad = 12750

lat_a=34.8300
long_a = -118.1275
lat_b = 34.7300
long_b = -117.9300


for x in range(0,6):
    temp = lat_a-(.2*x)
    folium.features.Circle(radius=rad,location=[temp, long_a], color ='gold',fill=False).add_to(m)
for x in range(0,5):
    folium.features.Circle(radius=rad,location=[lat_a-(.2*x), long_a+.4], color ='purple',fill=False).add_to(m)
for x in range(0,5):
    folium.features.Circle(radius=rad,location=[lat_a-(.2*x), long_a-.4], color ='blue',fill=False).add_to(m)
for x in range(0,5):
    if(x==4):
        folium.features.Circle(radius=rad,location=[lat_a-(.2*x), long_a-.8], color ='blue',fill=False).add_to(m)
    




for x in range(0,5):
    folium.features.Circle(radius=rad,location=[lat_b-(.2*x), long_b], color ='black',fill=False).add_to(m)
for x in range(0,6):
    folium.features.Circle(radius=rad,location=[lat_b-(.2*x), long_b-.4], color ='crimson',fill=False).add_to(m)
for x in range(0,4):
    if(x!=2):
        folium.features.Circle(radius=rad,location=[lat_b-(.2*x), long_b-.8], color ='green',fill=False).add_to(m)    

In [26]:
us_states = os.path.join('data', 'us-states.json')

geo_json_data = json.load(open(us_states))

In [27]:
import pandas as pd

US_Unemployment_Oct2012 = os.path.join('data', 'US_Unemployment_Oct2012.csv')
unemployment = pd.read_csv(US_Unemployment_Oct2012)

unemployment

,State,Unemployment
0,AL,7.1
1,AK,6.8
2,AZ,8.1
3,AR,7.2
4,CA,10.1
5,CO,7.7
6,CT,8.4
7,DE,7.1
8,FL,8.2
9,GA,8.8


In [28]:
from branca.colormap import linear

colormap = linear.YlGn.scale(
    unemployment.Unemployment.min(),
    unemployment.Unemployment.max())

print(colormap(5.0))

colormap

#c2e798


In [29]:
unemployment_dict = unemployment.set_index('State')['Unemployment']

unemployment_dict['AL']

7.0999999999999996

In [30]:
m = folium.Map([43, -100], zoom_start=4)

folium.GeoJson(
    geo_json_data,
    name='unemployment',
    style_function=lambda feature: {
        'fillColor': colormap(unemployment_dict[feature['id']]),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
).add_to(m)

folium.LayerControl().add_to(m)

m.save(os.path.join('results', 'GeoJSON_and_choropleth_5.html'))

m

In [31]:
m = folium.Map([43, -100], zoom_start=4)

m.choropleth(
    geo_data=us_states,
    data=unemployment,
    columns=['State', 'Unemployment'],
    key_on='feature.id',
    fill_color='YlGn',
    threshold_scale=[3, 4, 9, 10]
)


m.save(os.path.join('results', 'GeoJSON_and_choropleth_10.html'))

m